In [13]:
import numpy as np
import pandas as pd
import os
# Lista de posibles rutas
addresses = [
    'C:/Users/RONALD Q/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx',
    'C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx',
    'C:/Users/mticllacu/OneDrive - LUZ DEL SUR S.A.A/Archivos de Ronald Quispe Ocaña - ProyectoRyD_V2/Basededatos/RDIS.xlsx'
]

df = None
for path in addresses:
    if os.path.exists(path):   # verifica si existe
        df = pd.read_excel(path,header=1)
        print(f"✅ Archivo cargado desde: {path}")
        break

if df is None:
    raise FileNotFoundError("❌ No se encontró el archivo en ninguna de las rutas especificadas.")

✅ Archivo cargado desde: C:/Users/roquispec/OneDrive - LUZ DEL SUR S.A.A/Documentos/Estudios de Ingreso/ProyectoRyD_V2/Basededatos/RDIS.xlsx


### Limpieza

In [14]:
df["SERIE"] = df["SERIE"].astype(str)
df['SERIE'] = df['SERIE'].astype(str).str.replace(" ", "")
df = df.drop(columns=df.columns[0])
# Aseguramos que FECHA sea datetime
df["FECHA"] = pd.to_datetime(df["FECHA"], errors="coerce")
# df["SERIE"] = df["SERIE"].astype(str)

cap_cols = df.iloc[:,2:]

df.head()

,SERIE,FECHA,H-X DOWN [%] ONAF 2,H-X MID [%] ONAF 2,H-X TOP [%] ONAF 2,H-Y DOWN [%],H-Y MID [%],H-Y TOP [%],X-Y [%]
0,59571,2016-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,6.725,NaN,NaN,12.23
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Lógica

In [15]:
for col in cap_cols:
    # valor inicial por SERIE en la fecha mínima
    ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
    ref_mapped = df["SERIE"].map(ref)
     # variación porcentual
    delta = abs((df[col] - ref_mapped) / ref_mapped) * 100
    delta = delta.replace([np.inf, -np.inf], np.nan)

    # marcar casos inválidos
    mask = df[col].isna() | ref_mapped.isna() | (ref_mapped == 0) | (df[col] == 0)
    delta[mask] = np.nan

    # agregar columna nueva
    df[f"{col}_Delta"] = delta


# Calcular el máximo de las columnas que terminan en 'Delta' para cada fila
cols_delta = [col for col in df.columns if col.endswith('Delta')]
df['Max_Dis'] = df[cols_delta].max(axis=1)
df.head()

C:\Users\roquispec\AppData\Local\Temp\ipykernel_40424\2125369680.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Temp\ipykernel_40424\2125369680.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ref = df.groupby("SERIE").apply(lambda g: g.loc[g["FECHA"].idxmin(), col])
C:\Users\roquispec\AppData\Local\Tem

,SERIE,FECHA,H-X DOWN [%] ONAF 2,H-X MID [%] ONAF 2,H-X TOP [%] ONAF 2,H-Y DOWN [%],H-Y MID [%],H-Y TOP [%],X-Y [%],H-X DOWN [%] ONAF 2_Delta,H-X MID [%] ONAF 2_Delta,H-X TOP [%] ONAF 2_Delta,H-Y DOWN [%]_Delta,H-Y MID [%]_Delta,H-Y TOP [%]_Delta,X-Y [%]_Delta,Max_Dis
0,59571,2016-10-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,59571,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,59571,1996-04-16,NaN,NaN,NaN,6.725,NaN,NaN,12.23,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0
3,59572,2016-09-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,59572,2006-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Asignar Puntaje

In [16]:
# Asignar puntaje RDIS según el valor de Max_Dis y la tabla IEEE C57.152-2013
conditions = [
    df['Max_Dis'] > 3,
    (df['Max_Dis'] > 2) & (df['Max_Dis'] <= 3),
    df['Max_Dis'] <= 2
]
choices = [5, 3, 1]
df['RDIS'] = np.select(conditions, choices, default=np.nan)


In [17]:
# ---------------------------
# TABLAS FINALES Y FUNCIONES
# ---------------------------
# 1. Tabla con fechas originales: SERIE, FECHA, RDIS
df_DIS = df[['SERIE', 'FECHA', 'RDIS']].copy()

# 2. Tabla con fechas extendidas: SERIE, FECHA, RDIS
inicio = "2015-01-01"
fecha_inicio = pd.Timestamp(inicio)
fecha_fin = pd.Timestamp.today().normalize()
fechas = pd.date_range(fecha_inicio, fecha_fin, freq="D")
todas_series = df['SERIE'].dropna().unique()
calendario = pd.MultiIndex.from_product([todas_series, fechas], names=["SERIE", "FECHA"])
df_calendario = pd.DataFrame(index=calendario).reset_index()

ultimos_2014 = df_DIS[df_DIS['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2014['FECHA'] = fecha_inicio
base_ext = pd.concat([df_DIS, ultimos_2014], ignore_index=True)
df_DIS_ext = pd.merge(df_calendario, base_ext, on=["SERIE","FECHA"], how="left")
df_DIS_ext = df_DIS_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)

# 3. Tabla de detalles con fechas originales
cols_final = [col for col in df.columns if not col.endswith('Delta') and col != 'Max_Dis']
cols_final = ['SERIE', 'FECHA', 'RDIS'] + [col for col in cols_final if col not in ['SERIE', 'FECHA', 'RDIS']]
df_detalles = df[cols_final].copy()

# 4. Tabla de detalles con fechas extendidas
ultimos_2014_det = df_detalles[df_detalles['FECHA'] < fecha_inicio].sort_values('FECHA').groupby('SERIE').tail(1)
ultimos_2014_det['FECHA'] = fecha_inicio
base_ext_det = pd.concat([df_detalles, ultimos_2014_det], ignore_index=True)
df_detalles_ext = pd.merge(df_calendario, base_ext_det, on=["SERIE","FECHA"], how="left")
df_detalles_ext = df_detalles_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)


C:\Users\roquispec\AppData\Local\Temp\ipykernel_40424\4198853967.py:20: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_DIS_ext = df_DIS_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(drop=True)
C:\Users\roquispec\AppData\Local\Temp\ipykernel_40424\4198853967.py:32: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_detalles_ext = df_detalles_ext.groupby("SERIE").apply(lambda g: g.ffill()).reset_index(dro

In [18]:
# ---------------------------
# FUNCIONES PARA LLAMAR
# ---------------------------
def get_df_DIS():
    return df_DIS

def get_df_extendida_DIS():
    return df_DIS_ext

def get_df_detalles_DIS():
    return df_detalles

def get_df_detalles_ext_DIS():
    return df_detalles_ext

# ---------------------------
# PRINT DE TABLAS
# ---------------------------
print('\n ====== TABLA CON FECHAS ORIGINALES ====== \n')
print(get_df_DIS().head(20), '\n')
print('\n ====== TABLA CON FECHAS EXTENDIDAS ====== \n')
print(get_df_extendida_DIS().head(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== \n')
print(get_df_detalles_DIS().head(), '\n')
print('\n ====== TABLA DE DETALLES CON FECHAS EXTENDIDAS ====== \n')
print(get_df_detalles_ext_DIS().head(), '\n')


 ====== TABLA CON FECHAS ORIGINALES ====== 

     SERIE      FECHA  RDIS
0    59571 2016-10-23   NaN
1    59571 2006-01-29   NaN
2    59571 1996-04-16   1.0
3    59572 2016-09-18   NaN
4    59572 2006-01-29   NaN
5    59572 1996-04-22   1.0
6    59573 2016-09-18   NaN
7    59573 2006-01-29   NaN
8   146916 2019-06-30   NaN
9   146916 2015-01-06   NaN
10  146916 2014-11-12   1.0
11  146917 2022-05-12   NaN
12  146917 2014-12-05   1.0
13  146918 2019-07-11   5.0
14  146918 2014-11-25   1.0
15  185466 2015-11-22   NaN
16  185466 2014-09-12   NaN
17  185466 2005-10-21   1.0
18  185519 2020-06-16   NaN
19  185519 2016-07-10   NaN 


 ====== TABLA CON FECHAS EXTENDIDAS ====== 

     SERIE      FECHA  RDIS
0  123157T 2015-01-01   NaN
1  123157T 2015-01-02   NaN
2  123157T 2015-01-03   NaN
3  123157T 2015-01-04   NaN
4  123157T 2015-01-05   NaN 


 ====== TABLA DE DETALLES CON FECHAS ORIGINALES ====== 

   SERIE      FECHA  RDIS  H-X DOWN [%] ONAF 2  H-X MID [%] ONAF 2  \
0  59571 2016-10-23 